# Validating Data <a class="anchor" id="top"></a>

In this notebook, you will choose a dataset and prepare it for use with Amazon Personalize.

1. [How to Use the Notebook](#usenotebook)
1. [Introduction](#intro)
1. [Define your Use Case](#usecase)
1. [Choose a Dataset or Data Source](#source)
1. [Prepare the Interactions Data](#prepare_interactions)
1. [Prepare the Item Metadata](#prepare_items)
1. [Prepare the User Metadata](#prepare_users)
1. [Configure an S3 bucket and an IAM  role](#bucket_role)
1. [Create Dataset Group](#group_dataset)
1. [Create the Interactions Schema](#interact_schema)
1. [Create the Items (Movies) Schema](#items_schema)
1. [Create the Users Schema](#users_schema)
1. [Import the Interactions Data](#import_interactions)
1. [Import the Item Metadata](#import_items)
1. [Import the User Metadata](#import_users)

## How to Use the Notebook <a class="anchor" id="usenotebook"></a>

The code is broken up into cells like the one below. There's a triangular Run button at the top of this page that you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.

Simply follow the instructions below and execute the cells to get started with Amazon Personalize using case optimized recommenders.


## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

In Amazon Personalize, you start by creating a dataset group, which is a container for Amazon Personalize components. Your dataset group can be one of the following:

A Domain dataset group, where you create preconfigured resources for different business domains and use cases, such as getting recommendations for similar videos (VIDEO_ON_DEMAND domain) or best selling items (ECOMMERCE domain). You choose your business domain, import your data, and create recommenders. You use recommenders in your application to get recommendations.

Use a [Domain dataset group](https://docs.aws.amazon.com/personalize/latest/dg/domain-dataset-groups.html) if you have a video on demand or e-commerce application and want Amazon Personalize to find the best configurations for your use cases. If you start with a Domain dataset group, you can also add custom resources such as solutions with solution versions trained with recipes for custom use cases.

A [Custom dataset group](https://docs.aws.amazon.com/personalize/latest/dg/custom-dataset-groups.html), where you create configurable resources for custom use cases and batch recommendation workflows. You choose a recipe, train a solution version (model), and deploy the solution version with a campaign. You use a campaign in your application to get recommendations.

Use a Custom dataset group if you don't have a video on demand or e-commerce application or want to configure and manage only custom resources, or want to get recommendations in a batch workflow. If you start with a Custom dataset group, you can't associate it with a domain later. Instead, create a new Domain dataset group.

You can create and manage Domain dataset groups and Custom dataset groups with the AWS console, the AWS Command Line Interface (AWS CLI), or programmatically with the AWS SDKs.


## Define your Use Case <a class="anchor" id="usecase"></a>
[Back to top](#top)

There are a few guidelines for scoping a problem suitable for Personalize. We recommend the values below as a starting point, although the [official limits](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) lie a little lower.

* Authenticated users
* At least 50 unique users
* At least 100 unique items
* At least 2 dozen interactions for each user 

Most of the time this is easily attainable, and if you are low in one category, you can often make up for it by having a larger number in another category.

The user-item-iteraction data is key for getting started with the service. This means we need to look for use cases that generate that kind of data, a few common examples are:

1. Video-on-demand applications
1. E-commerce platforms

Defining your use-case will inform what data and what type of data you need.

In this example we are going to be creating:

1. Amazon Personalize VIDEO_ON_DEMAND Domain recommender for the ["More Like X"](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#more-like-y-use-case) use case.
1. Amazon Personalize VIDEO_ON_DEMAND Domain recommender for the ["Top pics for you"](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case) use case.
1. Amazon Personalize Custom Campaign for a personalized ranked list of movies, for instance shelf/rail/carousel based on some information (director, location, superhero franchise, etc...) 

All of these will be created within the same dataset group and with the same input data.

The diagram bellow shows an overview of what we will be building in this wokshop.

![Workflow](images/image1.png)

In this notebook we will be working on the Data Layer, Creatig a Dataset Group and importing the Datasets. 


## Choose a Dataset or Data Source <a class="anchor" id="source"></a>
[Back to top](#top)

Regardless of the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred

To begin, we are going to use the latest MovieLens dataset, this dataset has over 25 million interactions and a rich collection of metadata for items. There is also a smaller version of this dataset, which can be used to shorten training times, while still incorporating the same capabilities as the full dataset.

Generally speaking your data will not arrive in a perfect form for Personalize, and will take some modification to be structured correctly. This notebook guides you through all of that. 

Set USE_FULL_MOVIELENS to True to use the full dataset.

In [1]:
USE_FULL_MOVIELENS = False

First, you will download the dataset from the Movielens website and unzip it in a new folder using the code below.

In [2]:
data_dir = "poc_data"
!mkdir $data_dir

if not USE_FULL_MOVIELENS:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
    !cd $data_dir && unzip ml-latest-small.zip
    dataset_dir = data_dir + "/ml-latest-small/"
else:
    !cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-25m.zip
    !cd $data_dir && unzip ml-25m.zip
    dataset_dir = data_dir + "/ml-25m/"

--2022-10-15 10:45:01--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

100%[======================================>] 978,202     4.45MB/s   in 0.2s   

2022-10-15 10:45:01 (4.45 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


Take a look at the data files you have downloaded.

In [3]:
!ls $dataset_dir

links.csv  movies.csv  ratings.csv  README.txt	tags.csv


## Prepare the Interactions data <a class="anchor" id="prepare_interactions"></a>
[Back to top](#top)

The next thing to be done is to load the data and confirm the data is in a good state.

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK for python) and [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/)  which are core data science tools.

In [4]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd
import numpy as np

Next, open the `ratings.csv` file and take a look at the first rows.

In [5]:
original_data = pd.read_csv(dataset_dir + '/ratings.csv')
original_data.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
original_data.shape

(100836, 4)

In [7]:
original_data.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


This shows that we have a good range of values for `userId` and `movieId`. Next, it is always a good idea to confirm the data format.

In [8]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [9]:
original_data.isnull().any()

userId       False
movieId      False
rating       False
timestamp    False
dtype: bool

In [10]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

964982681.0
2000-07-30 18:44:41


From this, you can see that there are a total of (25,000,095 for full 100836 for small) entries in the dataset, with 4 columns, and each cell stored as int64 format, with the exception of the rating whihch is a float64.

The int64 format is clearly suitable for `userId` and `movieId`. However, we need to dive deeper to understand the timestamps in the data. To use Amazon Personalize, you need to save timestamps in [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time) format.

This date makes sense as a timestamp, so we can continue formatting the rest of the data. Remember, the data we need is user-item-interaction data, which is userId, movieId, and timestamp in this case. Our dataset has an additional column, rating, which can be dropped from the dataset after we have leveraged it to focus on positive interactions.

### Convert the Interactions Data

Since this is a dataset of an explicit feedback movie ratings, it includes movies rated from 1 to 5. We want to include only moves that were "liked" by the users, and simulate a dataset of data that would be gathered by a VOD platform. In order to do that, we will filter out all interactions under 2 out of 5, and create two event types: "Click" and and "Watch". We will then assign all movies rated 2 and above as "Click" and movies rated 4 and above as both "Click" and "Watch". 

Note that for a real data set you would actually model based on implicit feedback such as clicks, watches and/or explicit feedback such as ratings, likes etc.

In [11]:
watched_df = original_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'movieId', 'timestamp']]
watched_df['EVENT_TYPE']='Watch'
watched_df.head()

,userId,movieId,timestamp,EVENT_TYPE
0,1,1,964982703,Watch
1,1,3,964981247,Watch
2,1,6,964982224,Watch
3,1,47,964983815,Watch
4,1,50,964982931,Watch


In [12]:
clicked_df = original_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp']]
clicked_df['EVENT_TYPE']='Click'
clicked_df.head()

,userId,movieId,timestamp,EVENT_TYPE
0,1,1,964982703,Click
1,1,3,964981247,Click
2,1,6,964982224,Click
3,1,47,964983815,Click
4,1,50,964982931,Click


In [13]:
interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last') 

In [14]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158371 entries, 66679 to 81092
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   userId      158371 non-null  int64 
 1   movieId     158371 non-null  int64 
 2   timestamp   158371 non-null  int64 
 3   EVENT_TYPE  158371 non-null  object
dtypes: int64(3), object(1)
memory usage: 6.0+ MB


Lets look at what the new dataset looks like.

In [15]:
interactions_df.describe()

,userId,movieId,timestamp
count,158371.000000,158371.000000,1.583710e+05
mean,323.940804,19869.485146,1.211627e+09
std,182.195975,35809.058185,2.137376e+08
min,1.000000,1.000000,8.281246e+08
25%,175.000000,1203.000000,1.031072e+09
50%,325.000000,3033.000000,1.193289e+09
75%,475.000000,8528.000000,1.435998e+09
max,610.000000,193609.000000,1.537799e+09


 Amazon Personalize has default column names for users, items, and timestamp. These default column names are `USER_ID`, `ITEM_ID`, `TIMESTAMP` and `EVENT_VALUE` for the [VIDEO_ON_DEMAND domain dataset](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO-ON-DEMAND-datasets-and-schemas.html). The final modification to the dataset is to replace the existing column headers with the default headers.

In [16]:
interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

That's it! At this point the data is ready to go, and we just need to save it as a CSV file.

In [17]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Prepare the Item Metadata <a class="anchor" id="prepare_items"></a>
[Back to top](#top)

This will allow you to work with filters as well as supporting the [Top Pics for you Domain Recommender](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case), and complying with the [VIDEO_ON_DEMAND domain dataset and schema requirements](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO-ON-DEMAND-datasets-and-schemas.html#VIDEO-ON-DEMAND-dataset-requirements)..

Next we load the data and confirm the data is in a good state.

Next, open the `movies.csv` file and take a look at the first rows. This file has information about the movie.

In [18]:
original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [19]:
original_data.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


This does not really tell us much about the dataset, so we will explore a bit more and look at the raw information. We can see that genres often appear in groups. That is fine for us as Personalize supports this structure.

In [20]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [23]:
#DELETE ME AFTER IMDB DATA
original_data['year'] = original_data['title'].str.extract('.*\((.*)\).*',expand = False)
original_data.head(5)
original_data = original_data.dropna(axis=0)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


From this, you can see that there are a total of (62,000+ for full 9742 for small) entries in the dataset, with 3 columns.

Lets look for potential data issues. First we will check for null values.

In [25]:
original_data.isnull().sum()

movieId    0
title      0
genres     0
year       0
dtype: int64

Looks good, we currently have no null values.

From an item metadata perspective, we only want to include information that is relevant to training a model and/or filtering results, so we will drop the title column, and keep the genre information.

In [26]:
itemmetadata_df = original_data.copy()
itemmetadata_df = itemmetadata_df[['movieId', 'genres', 'year']]
itemmetadata_df.head()

,movieId,genres,year
0,1,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Adventure|Children|Fantasy,1995
2,3,Comedy|Romance,1995
3,4,Comedy|Drama|Romance,1995
4,5,Comedy,1995


We will add a new dataframe to help us generate a creation timestamp. If you don’t provide the CREATION_TIMESTAMP for an item, the model infers this information from the interaction dataset and uses the timestamp of the item’s earliest interaction as its corresponding release date. If an item doesn’t have an interaction, its release date is set as the timestamp of the latest interaction in the training set and it is considered a new item. 

For the current example we are selecting a today's date as the creation timestamp because the actual creation timestamp is unknown. In your use-case, please provide the appropriate creation timestamp for the item. This can be when the item was added to your platform.

In [27]:
ts = datetime(2022, 1, 1, 0, 0).strftime('%s')
print(ts)

itemmetadata_df['CREATION_TIMESTAMP'] = ts

1640995200


After manipulating the data, always confirm that the data format has not changed.

In [28]:
itemmetadata_df.dtypes

movieId                int64
genres                object
year                  object
CREATION_TIMESTAMP    object
dtype: object

Amazon Personalize has a default column for `ITEM_ID` that will map to our `movieId`. We will flesh out more information by specifying `GENRE` as well.

In [29]:
itemmetadata_df.rename(columns = {'genres':'GENRES', 'movieId':'ITEM_ID', 'year':'YEAR'}, inplace = True) 

In [30]:
itemmetadata_df

,ITEM_ID,GENRES,YEAR,CREATION_TIMESTAMP
0,1,Adventure|Animation|Children|Comedy|Fantasy,1995,1640995200
1,2,Adventure|Children|Fantasy,1995,1640995200
2,3,Comedy|Romance,1995,1640995200
3,4,Comedy|Drama|Romance,1995,1640995200
4,5,Comedy,1995,1640995200
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,2017,1640995200
9738,193583,Animation|Comedy|Fantasy,2017,1640995200
9739,193585,Drama,2017,1640995200
9740,193587,Action|Animation,2018,1640995200


That's it! At this point the item data is ready to go, and we just need to save it as a CSV file.

In [31]:
items_filename = "item-meta.csv"
itemmetadata_df.to_csv((data_dir+"/"+items_filename), index=False, float_format='%.0f')

## Prepare the User Metadata <a class="anchor" id="prepare_users"></a>
[Back to top](#top)

This will supporting the [Top Pics for you Domain Recommender](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO_ON_DEMAND-use-cases.html#top-picks-use-case), and complying with the [VIDEO_ON_DEMAND domain dataset and schema requirements](https://docs.aws.amazon.com/personalize/latest/dg/VIDEO-ON-DEMAND-datasets-and-schemas.html#VIDEO-ON-DEMAND-dataset-requirements).

The dataset does not have any user metadata so we will create a fake metadata field.

In [32]:
# get all unique user ids from the interaction dataset

user_ids = interactions_df['USER_ID'].unique()
user_data = pd.DataFrame()
user_data["USER_ID"]= user_ids
user_data

,USER_ID
0,429
1,107
2,191
3,99
4,54
...,...
605,77
606,25
607,596
608,184


Adding Metadata
The current dataset does not contain additiona user information. For this example, we'll randomly assign a membership level. For Ad Supported models this could indicated premium vs ad supported.

In [33]:
possible_membership_levels = ['silver', 'gold']
random = np.random.choice(possible_membership_levels, len(user_data.index), p=[0.5, 0.5])
user_data["MEMBERLEVEL"] = random
user_data

,USER_ID,GENDER
0,429,female
1,107,male
2,191,male
3,99,male
4,54,male
...,...,...
605,77,male
606,25,male
607,596,male
608,184,female


In [34]:
# Saving the data as a CSV file
users_filename = "users.csv"
user_data.to_csv((data_dir+"/"+users_filename), index=False, float_format='%.0f')


We will use this data in the next labs. In order to use this data we will store these variables so subsequent notebooks can use this data. After completing that cell open notebook `02_Training_Layer.ipynb` to continue.

In [35]:
%store USE_FULL_MOVIELENS
%store dataset_dir
%store data_dir
%store interactions_filename
%store items_filename
%store users_filename

Stored 'USE_FULL_MOVIELENS' (bool)
Stored 'dataset_dir' (str)
Stored 'data_dir' (str)
Stored 'interactions_filename' (str)
Stored 'items_filename' (str)
Stored 'users_filename' (str)
